To switch to dbt profile with the `arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo` role that I created

In [ ]:
nano ~/.aws/config

# In nano add this profile
[profile dbt-glue]
role_arn = arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo
source_profile = Joshua_Harris_ETL
region = ap-southeast-2

# switch profile for the terminal session
export AWS_PROFILE=dbt-glue

# check your cureent profile
aws sts get-caller-identity

lets make a csv file with example data

In [92]:
# pip install pandas numpy

In [93]:
from pandas import DataFrame
from numpy import nan

import numpy as np

def permute_dataframe(df, random_state=None):
    """
    Returns a new DataFrame with the rows randomly permuted.
    Each call produces a different order unless random_state is set.
    """
    return df.sample(frac=1, random_state=random_state).reset_index(drop=True)


dirty_df = DataFrame({
    'patient_id': ["A001", "A002", "A003", "A004", "A005", ""], 
    'sex': ["M", "F", "0", "1", "Male", "fem"],
    'medication': [None, '', None, nan, None, 'statin'],
    'age': [nan, 32, 214, 21, 0, ""]
    })


# Example usage of permute_dataframe:

import random
permuted_df = permute_dataframe(dirty_df, random_state=random.randint(0, 1_000_000))


with open('../data/patient_info.csv', 'w') as f:
    permuted_df.to_csv(f, index=False)

lets upload this file to s3

In [94]:
%%bash
DATETIME=$(date +%Y%m%d%H%M%S)
DATA_NAME="patient_info"
aws s3 cp "../data/patient_info.csv" "s3://acdc-dbt-test-raw/data/${DATA_NAME}/date_time=${DATETIME}/${DATA_NAME}.csv" --profile "default"

upload: ../data/patient_info.csv to s3://acdc-dbt-test-raw/data/patient_info/date_time=20250725135117/patient_info.csv


lets make some other data types

In [95]:
from pandas import DataFrame
from numpy import nan

dirty_df = DataFrame({
    'sample_id': ["SAM0001", "SAM0002", "SAM0003", "SAM0004", "SAM0005", ""],
    'patient_id': ["A001", "A002", "A003", "A004", "A005", ""], 
    'storage_medium': ["nitrogen", "air", "nitrogen", "air", "nitrogen", "air"],
    'storage_date': ["2022-01-01", "2022-01-02", "2022-01-03", "2022-01-04", "2022-01-05", "2022-01-06"],
    'volume_ul': [0, 486, 4, 4000, 5000, 6000]
    })

permuted_df = permute_dataframe(dirty_df, random_state=random.randint(0, 1_000_000))

with open('../data/sample.csv', 'w') as f:
    permuted_df.to_csv(f, index=False)

In [96]:
%%bash
DATETIME=$(date +%Y%m%d%H%M%S)
DATA_NAME="sample"
aws s3 cp "../data/sample.csv" "s3://acdc-dbt-test-raw/data/${DATA_NAME}/date_time=${DATETIME}/${DATA_NAME}.csv" --profile "default"

upload: ../data/sample.csv to s3://acdc-dbt-test-raw/data/sample/date_time=20250725135118/sample.csv


# Trying to run dbt

### IAM roles:

The principle role: `arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo` as the following policies attached:
-  `arn:aws:iam::026090528544:policy/acdc-dbt-glue-test`
-  `arn:aws:iam::026090528544:policy/GlueInteractiveSessionClientPolicy-dbt`
        - This role Grants permission to create and manage AWS Glue interactive sessions and pass the designated execution role required to run dbt workloads in AWS Glue.

In [ ]:
# Checking dbt status to see if it connected and ready
!dbt debug # Run in terminal

Should look like:

```
dbt debug                                                                                                                                                                                                                                                                                                            (main✱) 
04:33:16  Running with dbt=1.9.8
04:33:16  dbt version: 1.9.8
04:33:16  python version: 3.9.5
04:33:16  python path: /Users/harrijh/.pyenv/versions/3.9.5/bin/python3.9
04:33:16  os info: macOS-15.5-arm64-arm-64bit
04:33:16  Using profiles dir at /Users/harrijh/projects/acdc-datapipeline-demo/dbt_demo_acdc
04:33:16  Using profiles.yml file at /Users/harrijh/projects/acdc-datapipeline-demo/dbt_demo_acdc/profiles.yml
04:33:16  Using dbt_project.yml file at /Users/harrijh/projects/acdc-datapipeline-demo/dbt_demo_acdc/dbt_project.yml
04:33:16  adapter type: glue
04:33:16  adapter version: 1.9.4
04:33:16  Configuration:
04:33:16    profiles.yml file [OK found and valid]
04:33:16    dbt_project.yml file [OK found and valid]
04:33:16  Required dependencies:
04:33:16   - git [OK found]

04:33:16  Connection:
04:33:16    role_arn: arn:aws:iam::026090528544:role/AwsGlueSessionUserRestrictedServiceRole-dbtGlueExecutor
04:33:16    region: ap-southeast-2
04:33:16    workers: 2
04:33:16    worker_type: G.1X
04:33:16    session_provisioning_timeout_in_seconds: 120
04:33:16    schema: glue-db-acdc-dbt-test-clean
04:33:16    location: s3://acdc-dbt-test-clean/dbt/
04:33:16    extra_jars: None
04:33:16    idle_timeout: 10
04:33:16    query_timeout_in_minutes: 300
04:33:16    glue_version: 5.0
04:33:16    security_configuration: None
04:33:16    connections: None
04:33:16    conf: None
04:33:16    extra_py_files: None
04:33:16    delta_athena_prefix: None
04:33:16    tags: None
04:33:16    seed_format: parquet
04:33:16    seed_mode: overwrite
04:33:16    default_arguments: None
04:33:16    iceberg_glue_commit_lock_table: myGlueLockTable
04:33:16    use_interactive_session_role_for_api_calls: False
04:33:16    lf_tags: None
04:33:16    glue_session_id: None
04:33:16    glue_session_reuse: True
04:33:16    datalake_formats: None
04:33:16    enable_session_per_model: False
04:33:16    use_arrow: False
04:33:16    enable_spark_seed_casting: False
04:33:16  Registered adapter: glue=1.9.4
04:33:50    Connection test: [OK connection ok]

04:33:50  All checks passed!
```